In [261]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [289]:
import pandas as pd

In [290]:
# connect to the MySQL database
import mysql.connector
import os

try:
    database_connect = mysql.connector.connect(
        host='localhost',
        user='root',
        password=os.getenv('MYSQL_PASSWORD'),
        database='art'
    )
    print("Successfully connected to MySQL!")

    cursor = database_connect.cursor()

except mysql.connector.Error as err:
    print(f"Error: {err}")

Successfully connected to MySQL!


In [291]:
# list of CSV files and their corresponding table names
csv_files = [
    ('work.csv', 'work'),
    ('artist.csv', 'artist'),
    ('canvas_size.csv', 'canvas_size'),
    ('museum.csv', 'museum'),
    ('museum_hours.csv', 'museum_hours'),
    ('product_size.csv', 'product_size'),
    ('subject.csv', 'subject')
]

In [362]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
cursor.execute("DROP TABLE IF EXISTS artist, canvas_size, museum, work, museum_hours, subject, product_size")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
database_connect.commit()

In [363]:
query = "SHOW Tables;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

In [364]:
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = csv_file
    
    # read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    for col in df.columns:
        if df[col].dtype == 'object':  # check if column is detected as text
            try:
                df[col] = pd.to_numeric(df[col])  # convert to numeric where possible
            except ValueError:
                pass  

    
    # fill Nan's with None for SQL to mark as Null
    df = df.astype(object).where(pd.notnull(df), None)
    
    # show the number of missing values found
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)
        
    # commit the transaction for the current CSV file
    database_connect.commit()

Processing work.csv
NaN values before replacement:
work_id          0
name             0
artist_id        0
style         1286
museum_id    10223
dtype: int64

Processing artist.csv
NaN values before replacement:
artist_id         0
full_name         0
first_name        0
middle_names    273
last_name         0
nationality       0
style             0
birth             0
death             0
dtype: int64

Processing canvas_size.csv
NaN values before replacement:
size_id    0
width      0
height     7
label      0
dtype: int64

Processing museum.csv
NaN values before replacement:
museum_id     0
name          0
address       0
city          0
state        19
postal        7
country       0
phone         0
url           0
dtype: int64

Processing museum_hours.csv
NaN values before replacement:
museum_id    0
day          0
open         0
close        0
dtype: int64

Processing product_size.csv
NaN values before replacement:
work_id          0
size_id          0
sale_price       0
regular_p

## Data Preprocessing

#### Work Table

In [365]:
# query
query = "SELECT * FROM work LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,work_id,name,artist_id,style,museum_id
0,160228,Still Life with Flowers and a Watch,615,Baroque,43.0
1,160236,Still Life with Fruit and a Beaker on a Cock's...,615,Baroque,43.0
2,160244,Still Life with Fruit and a Goldfinch,615,Baroque,43.0
3,160252,Still Life with Fruit and Oysters,615,Baroque,43.0
4,160260,"Still Life with Fruit, Oysters, and a Porcelai...",615,Baroque,43.0


In [366]:
# change numerical data types from text
alter_query = """
ALTER TABLE work
MODIFY COLUMN work_id INT,
MODIFY COLUMN artist_id INT,
MODIFY COLUMN museum_id INT;
"""

cursor.execute(alter_query)
database_connect.commit()

#### Artist Table

In [367]:
# query
query = "SELECT * FROM artist LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,artist_id,full_name,first_name,middle_names,last_name,nationality,style,birth,death
0,500,Pierre-Auguste Renoir,Pierre,Auguste,Renoir,French,Impressionist,1841,1919
1,501,Alexandre Cabanel,Alexandre,None,Cabanel,French,Classicist,1823,1889
2,502,James Ensor,James,None,Ensor,Belgian,Expressionist,1860,1949
3,503,Maximilien Luce,Maximilien,None,Luce,French,Pointillist,1858,1941
4,504,August Macke,August,None,Macke,German,Expressionist,1887,1914


In [368]:
query = "DESC artist;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('artist_id', 'text', 'YES', '', None, '')
('full_name', 'text', 'YES', '', None, '')
('first_name', 'text', 'YES', '', None, '')
('middle_names', 'text', 'YES', '', None, '')
('last_name', 'text', 'YES', '', None, '')
('nationality', 'text', 'YES', '', None, '')
('style', 'text', 'YES', '', None, '')
('birth', 'text', 'YES', '', None, '')
('death', 'text', 'YES', '', None, '')


In [369]:
# change numerical data types from text
alter_query = """
ALTER TABLE artist
MODIFY COLUMN artist_id INT,
MODIFY COLUMN birth INT,
MODIFY COLUMN death INT;
"""

cursor.execute(alter_query)
database_connect.commit()

#### Canvas_size Table

In [370]:
# query
query = "SELECT * FROM canvas_size LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,size_id,width,height,label
0,20,20,None,"20"" Long Edge"
1,24,24,None,"24"" Long Edge"
2,30,30,None,"30"" Long Edge"
3,36,36,None,"36"" Long Edge"
4,40,40,None,"40"" Long Edge"


In [371]:
query = "DESC canvas_size;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('size_id', 'text', 'YES', '', None, '')
('width', 'text', 'YES', '', None, '')
('height', 'text', 'YES', '', None, '')
('label', 'text', 'YES', '', None, '')


In [372]:
# change numerical data types from text
alter_query = """
ALTER TABLE canvas_size
MODIFY COLUMN size_id INT,
MODIFY COLUMN width INT,
MODIFY COLUMN height INT;
"""

cursor.execute(alter_query)
database_connect.commit()

#### Museum Table

In [373]:
# query
query = "SELECT * FROM museum LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,museum_id,name,address,city,state,postal,country,phone,url
0,30,The Museum of Modern Art,11 W 53rd St,New York,NY,10019,USA,+1 212 708-9400,https://www.moma.org/
1,31,Pushkin State Museum of Fine Arts,12 Ulitsa Volkhonka,Moscow,None,119019,Russia,+7 495 697-95-78,https://pushkinmuseum.art/
2,32,National Gallery of Victoria,180 St Kilda Rd,Melbourne,Victoria,3004,Australia,+61 (0)3 8620 2222,https://www.ngv.vic.gov.au/
3,33,São Paulo Museum of Art,"Av. Paulista, 1578 - Bela Vista",São Paulo,None,01310-200,Brazil,+55 11 3149-5959,https://masp.org.br/
4,34,The State Hermitage Museum,Palace Square,2,Sankt-Peterburg,190000,Russia,7 812 710-90-79,https://www.hermitagemuseum.org/wps/portal/her...


In [374]:
query = "DESC museum;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('museum_id', 'text', 'YES', '', None, '')
('name', 'text', 'YES', '', None, '')
('address', 'text', 'YES', '', None, '')
('city', 'text', 'YES', '', None, '')
('state', 'text', 'YES', '', None, '')
('postal', 'text', 'YES', '', None, '')
('country', 'text', 'YES', '', None, '')
('phone', 'text', 'YES', '', None, '')
('url', 'text', 'YES', '', None, '')


In [375]:
# change numerical data types from text
alter_query = """
ALTER TABLE museum
MODIFY COLUMN museum_id INT
"""

cursor.execute(alter_query)
database_connect.commit()

#### Museum_hours Table

In [376]:
# query
query = "SELECT * FROM museum_hours LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,museum_id,day,open,close
0,30,Sunday,10:30:AM,05:30:PM
1,30,Monday,10:30:AM,05:30:PM
2,30,Tuesday,10:30:AM,05:30:PM
3,30,Wednesday,10:30:AM,05:30:PM
4,30,Thusday,10:30:AM,05:30:PM


In [377]:
query = "DESC museum_hours;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('museum_id', 'text', 'YES', '', None, '')
('day', 'text', 'YES', '', None, '')
('open', 'text', 'YES', '', None, '')
('close', 'text', 'YES', '', None, '')


In [378]:
# add temporary columns to convert open and close columns to 24-hr time
alter_query = """
ALTER TABLE museum_hours
ADD COLUMN open_new TIME,
ADD COLUMN close_new TIME;
"""

cursor.execute(alter_query)
database_connect.commit()

In [379]:
# convert original open and close columns from text to time in new columns
update_query = """
UPDATE museum_hours
SET open_new = STR_TO_DATE(open, '%h:%i:%p'),
    close_new = STR_TO_DATE(close, '%h:%i:%p')
"""

cursor.execute(update_query)
database_connect.commit()

In [380]:
# drop old columns and rename new columns
drop_query = """
ALTER TABLE museum_hours
DROP COLUMN open,
DROP COLUMN close;
"""

cursor.execute(drop_query)
database_connect.commit()

In [381]:
# drop old columns and rename new columns
drop_query = """
ALTER TABLE museum_hours
CHANGE COLUMN open_new open TIME,
CHANGE COLUMN close_new close TIME;
"""

cursor.execute(drop_query)
database_connect.commit()

In [382]:
# change numerical data types from text
alter_query = """
ALTER TABLE museum_hours
MODIFY COLUMN museum_id INT,
MODIFY COLUMN day VARCHAR(20) NOT NULL;
"""

cursor.execute(alter_query)
database_connect.commit()

#### Product_Size Table

In [383]:
# query
query = "SELECT * FROM product_size LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,work_id,size_id,sale_price,regular_price
0,160228,24.0,85,85
1,160228,30.0,95,95
2,160236,24.0,85,85
3,160236,30.0,95,95
4,160244,24.0,85,85


In [384]:
query = "DESC product_size;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('work_id', 'text', 'YES', '', None, '')
('size_id', 'text', 'YES', '', None, '')
('sale_price', 'text', 'YES', '', None, '')
('regular_price', 'text', 'YES', '', None, '')


In [385]:
# change numerical data types from text
alter_query = """
ALTER TABLE product_size
MODIFY COLUMN work_id INT,
MODIFY COLUMN size_id INT,
MODIFY COLUMN sale_price DECIMAL(10,2),
MODIFY COLUMN regular_price DECIMAL(10,2)
;
"""

cursor.execute(alter_query)
database_connect.commit()

#### Subject Table

In [386]:
# query
query = "SELECT * FROM subject LIMIT 5;"
# run SQL query
cursor.execute(query)
# fetch data from query
data = cursor.fetchall()
# fetch column names
columns = [desc[0] for desc in cursor.description]
# convert to df
df = pd.DataFrame(data, columns=columns)
# display first few rows of df
df.head()

,work_id,subject
0,160228,Still-Life
1,160236,Still-Life
2,160244,Still-Life
3,160252,Still-Life
4,160260,Still-Life


In [387]:
query = "DESC subject;"
cursor.execute(query)

# fetch and print results
for row in cursor.fetchall():
    print(row)

('work_id', 'text', 'YES', '', None, '')
('subject', 'text', 'YES', '', None, '')


In [388]:
# change numerical data types from text
alter_query = """
ALTER TABLE subject
MODIFY COLUMN work_id INT,
MODIFY COLUMN subject VARCHAR(100) NOT NULL
;
"""

cursor.execute(alter_query)
database_connect.commit()

### Remove Duplicates

In [389]:
# check for potential duplicates in work table
query = """
SELECT work_id, name,  COUNT(*)
FROM work
GROUP BY work_id, name
HAVING COUNT(*)>1
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results[:10]:
    print(row)

(122691, 'Landscape with a Sunlit Stream', 2)
(122662, 'The Dunes at Camiers', 2)
(181576, 'Black and White and Grey', 2)
(181403, 'Black Stripe, Red Stripe', 2)
(181585, 'Black, White Stripe, Green On Pink', 2)
(181791, 'Blue and Pinks', 2)
(181356, 'Blue, Dark Blue and Orange', 2)
(181855, 'Blue, Fuschia Red and Pink', 2)
(181331, 'Blue, Orange and Gold', 2)
(181813, 'Blue Shades on Light Brown', 2)


In [390]:
# remove duplicates in work table
alter_query="""
CREATE TABLE work_deduplicated AS
SELECT DISTINCT * FROM work
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
DROP TABLE work
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
ALTER TABLE work_deduplicated RENAME TO work
;
"""
cursor.execute(alter_query)
database_connect.commit()

In [391]:
# check for potential duplicates in museum_hours table
query= """
SELECT museum_id, day, open, close, COUNT(*) AS duplicate_count
FROM museum_hours
GROUP BY museum_id, day, open, close
HAVING COUNT(*) > 1;
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results:
    print(row)

(80, 'Saturday', datetime.timedelta(seconds=39600), datetime.timedelta(seconds=61200), 2)


In [392]:
# remove duplicates in museum_hours table
alter_query="""
CREATE TABLE museum_hours_deduplicated AS
SELECT DISTINCT * FROM museum_hours
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
DROP TABLE museum_hours
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
ALTER TABLE museum_hours_deduplicated RENAME TO museum_hours
;
"""
cursor.execute(alter_query)
database_connect.commit()

In [393]:
# check for potential duplicates in subject table
query= """
SELECT work_id, subject, COUNT(*)
FROM subject
GROUP BY work_id, subject
HAVING COUNT(*) > 1;
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results[:10]:
    print(row)

(122662, 'Seascapes', 2)
(181576, 'Abstract/Modern Art', 2)
(181403, 'Abstract/Modern Art', 2)
(181585, 'Abstract/Modern Art', 2)
(181791, 'Abstract/Modern Art', 2)
(181356, 'Abstract/Modern Art', 2)
(181855, 'Abstract/Modern Art', 2)
(181331, 'Abstract/Modern Art', 2)
(181813, 'Abstract/Modern Art', 2)
(181941, 'Abstract/Modern Art', 2)


In [394]:
# remove duplicates in subject table
alter_query="""
CREATE TABLE subject_deduplicated AS
SELECT DISTINCT * FROM subject
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
DROP TABLE subject
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
ALTER TABLE subject_deduplicated RENAME TO subject
;
"""
cursor.execute(alter_query)
database_connect.commit()

In [395]:
# check for works that have more than one subject in subject table
query= """
SELECT s.work_id, s.subject
FROM subject s
JOIN(
    SELECT work_id
    FROM subject
    GROUP BY work_id
    HAVING COUNT(*)>1)
duplicates ON s.work_id = duplicates.work_id
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results[:10]:
    print(row)

(147731, 'Musics')
(147731, 'Portraits')
(147775, 'Musics')
(147775, 'Portraits')
(147786, 'Dancers')
(147786, 'Portraits')
(24536, 'Portraits')
(24536, 'U.S. Presidents')
(122171, 'Dogs')
(122171, 'Portraits')


In [405]:
# check in product size for duplicates
query= """
SELECT work_id, size_id, COUNT(*)
FROM product_size
GROUP BY work_id, size_id
HAVING COUNT(*) >1
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results:
    print(row)

(160228, 24, 2)
(160228, 30, 2)
(160236, 24, 2)
(160236, 30, 2)
(160244, 24, 2)
(160244, 30, 2)
(160252, 24, 2)
(160252, 30, 2)
(160260, 24, 2)
(160260, 30, 2)
(160268, 24, 2)
(160268, 30, 2)
(125752, 30, 2)
(125752, 24, 2)
(125752, 3024, 2)
(125752, 3226, 2)
(125752, 3629, 2)
(125752, 4030, 2)
(125752, 4836, 2)
(125752, 6048, 2)
(125752, 6854, 2)
(125818, 30, 2)
(125818, 24, 2)
(125818, 3624, 2)
(125818, 4630, 2)
(125818, 5436, 2)
(125818, 6040, 2)
(125818, 7248, 2)
(23448, 20, 2)
(23448, 24, 2)
(23448, 30, 2)
(23448, 3024, 2)
(23448, 3226, 2)
(23448, 3629, 2)
(23448, 4030, 2)
(23448, 4836, 2)
(23448, 6048, 2)
(23448, 6854, 2)
(125763, 30, 2)
(125763, 24, 2)
(125763, 3024, 2)
(125763, 3226, 2)
(125763, 3629, 2)
(125763, 4030, 2)
(125763, 4836, 2)
(125763, 6048, 2)
(125763, 6854, 2)
(125774, 30, 2)
(125774, 24, 2)
(125774, 3024, 2)
(125774, 3226, 2)
(125774, 3629, 2)
(125774, 4030, 2)
(125774, 4836, 2)
(125774, 6048, 2)
(125774, 6854, 2)
(125785, 30, 2)
(125785, 24, 2)
(125785, 3024, 2

In [406]:
# remove duplicates in product_size table
alter_query="""
CREATE TABLE product_size_deduplicated AS
SELECT DISTINCT * FROM product_size
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
DROP TABLE product_size
;
"""
cursor.execute(alter_query)
database_connect.commit()

alter_query="""
ALTER TABLE product_size_deduplicated RENAME TO product_size
;
"""
cursor.execute(alter_query)
database_connect.commit()

In [411]:
# check in for artworks in product_size that have a size_id not listed in canvas_size
query= """
SELECT size_id
FROM product_size
WHERE size_id NOT IN (SELECT size_id from canvas_size)
;
"""
cursor.execute(query)

# fetch and print results
results = cursor.fetchall()
for row in results[:10]:
    print(row)

(16,)
(29,)
(32,)
(32,)
(37,)
(27,)
(32,)
(29,)
(58,)
(22,)


In [412]:
# delete rows in product_size which have size_id not listed in canvas_size
delete_query = """
DELETE FROM product_size
WHERE size_id NOT IN (SELECT size_id FROM canvas_size);
"""
cursor.execute(delete_query)
database_connect.commit()

### Create Primary and Foreign Keys

In [396]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE artist 
ADD PRIMARY KEY (artist_id)
;
"""

cursor.execute(alter_query)
database_connect.commit()

In [397]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE canvas_size 
ADD PRIMARY KEY (size_id)
;
"""

cursor.execute(alter_query)
database_connect.commit()

In [398]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE museum 
ADD PRIMARY KEY (museum_id)
;
"""

cursor.execute(alter_query)
database_connect.commit()

In [399]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE work 
ADD PRIMARY KEY (work_id),
ADD CONSTRAINT fk_work_artist FOREIGN KEY (artist_id) REFERENCES artist(artist_id) ON DELETE CASCADE,
ADD CONSTRAINT fk_work_museum FOREIGN KEY (museum_id) REFERENCES museum(museum_id) ON DELETE CASCADE
;
"""

cursor.execute(alter_query)
database_connect.commit()

In [400]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE museum_hours 
ADD PRIMARY KEY (museum_id, day),
ADD CONSTRAINT fk_museum_hours FOREIGN KEY (museum_id) REFERENCES museum(museum_id) ON DELETE CASCADE;
"""

cursor.execute(alter_query)
database_connect.commit()

In [401]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE subject 
ADD PRIMARY KEY (work_id, subject),
ADD CONSTRAINT fk_subject_work FOREIGN KEY (work_id) REFERENCES work(work_id) ON DELETE CASCADE;
"""

cursor.execute(alter_query)
database_connect.commit()

In [413]:
# add primary and foreign keys for table
alter_query="""
ALTER TABLE product_size
ADD PRIMARY KEY (work_id, size_id),
ADD CONSTRAINT fk_product_work FOREIGN KEY (work_id) REFERENCES work(work_id) ON DELETE CASCADE,
ADD CONSTRAINT fk_product_canvas FOREIGN KEY (size_id) REFERENCES canvas_size(size_id) ON DELETE CASCADE
;
"""

cursor.execute(alter_query)
database_connect.commit()